因为股票会有分红派息，因此直接拿价格做回测是不准确的，我们需要对其进行复权处理。复权包含两种方式：

前复权：最后一天的价格不变，根据分红除权数据处理之前的价格数据；绝大多数股票中默认的都是向前复权，因为这样，最新价是实际价，不影响看盘；

后复权：上市第一天的价格不变，根据分红配股数据处理之后的价格数据，因此会导致最后一天的价格显示出来不是实际成交价。如果计算收益率，则后复权更准确，前复权有偏差。例如经常分红的基金，前复权会导致负数出现。

因此，一般采用后复权，这样计算的收益率才是相对正确的，而且查看收益率也很直观。后复权唯一不好的一点正如楼上所述：如果和实盘对接，需要根据复权因子转化一下。

In [2]:
from config import token
import tushare as ts
import pandas as pd 
from sqlalchemy import create_engine
from datetime import datetime
ts.set_token(token)

In [9]:
# get the datetime
now = datetime.now()
syear = str(now.year)
if now.month < 10:
    smonth = '0' + str(now.month)
else:
    smonth = str(now.month)
if now.day < 10:
    sday = '0' + str(now.day)
else:
    sday = str(now.day)
end_date = syear + smonth + sday
end_date

'20220914'

In [22]:
# get ticker name
ticker_engine = create_engine('sqlite:///../../dataset/ch/ticker_symbol_ch.db')
ticker_df = pd.read_sql('TOTAL', con=ticker_engine)
ticker_list = ticker_df.ts_code.to_list()

price_engine = create_engine('sqlite:///../../dataset/ch/ch_ticker_price_hfq.db')

In [ ]:
# 

for ticker in ticker_list:
    df = ts.pro_bar(ts_code=ticker, adj='hfq', start_date='20150101', end_date=end_date)
    df.trade_date = df.trade_date.apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])
    df = df[::-1]
    df.index = range(len(df))
    df.to_sql(ticker, con=price_engine,if_exists='replace',index=None)


In [23]:
ticker='000001.SZ'
df = ts.pro_bar(ts_code=ticker, adj='hfq', start_date='20150101', end_date=end_date)
df.trade_date = df.trade_date.apply(lambda x: str(x)[:4] + '-' + str(x)[4:6] + '-' + str(x)[6:])
df = df[::-1]
df.index = range(len(df))
df.to_sql(ticker, con=price_engine,if_exists='replace',index=None)

1874

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
1873,000001.SZ,2015-01-05,1136.1535,1156.7591,1108.4424,1138.2851,1125.4954,12.7897,1.1364,2860436.43,4.565388e+06
1872,000001.SZ,2015-01-06,1126.2059,1164.5751,1104.8897,1121.2321,1138.2851,-17.0530,-1.4981,2166421.40,3.453446e+06
1871,000001.SZ,2015-01-07,1105.6002,1124.7848,1087.1262,1099.9159,1121.2321,-21.3162,-1.9011,1700120.67,2.634796e+06
1870,000001.SZ,2015-01-08,1101.3370,1106.3108,1058.7046,1062.9678,1099.9159,-36.9481,-3.3592,1407714.21,2.128003e+06
1869,000001.SZ,2015-01-09,1058.7046,1127.6270,1045.2043,1071.4943,1062.9678,8.5265,0.8021,2508500.23,3.835378e+06
...,...,...,...,...,...,...,...,...,...,...,...
4,000001.SZ,2022-09-06,1433.3174,1442.4323,1416.2270,1424.2025,1432.1780,-7.9755,-0.5569,731294.99,9.146922e+05
3,000001.SZ,2022-09-07,1415.0876,1415.0876,1399.1365,1404.8333,1424.2025,-19.3692,-1.3600,979812.81,1.208330e+06
2,000001.SZ,2022-09-08,1403.6940,1417.3663,1401.4153,1408.2514,1404.8333,3.4181,0.2433,621116.92,7.689135e+05
1,000001.SZ,2022-09-09,1412.8089,1451.5472,1408.2514,1449.2685,1408.2514,41.0171,2.9126,1951297.31,2.469131e+06
